In [1]:
import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from skimage.measure import find_contours

from itertools import izip
import pandas as pd

In [2]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [3]:
mesh_rootdir = '/home/yuncong/CSHL_meshes'
if not os.path.exists(mesh_rootdir):
    os.makedirs(mesh_rootdir)

In [23]:
# stack = 'MD593'

# for stack in ['MD589', 'MD593']:
for stack in ['MD593']:
    
    volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax = \
        np.loadtxt('/home/yuncong/CSHL_volumes/volume_' + stack + '_annotation_limits.txt')
        
    _, voxel_size_xy, section_zdistance = np.loadtxt('/home/yuncong/CSHL_volumes/volume_' + stack + '_annotation_resolution.txt')
    
    annotation_zlimits = pd.read_hdf('/home/yuncong/Brain/database/'+stack+'_label_polygons.h5', 'zlimits').to_dict()
    
    mesh_dir = os.path.join(mesh_rootdir, stack)
    if not os.path.exists(mesh_dir):
        os.makedirs(mesh_dir)

    data_matrix_int = bp.unpack_ndarray_file('/home/yuncong/CSHL_volumes/volume_' + stack + '_annotation.bp')
    ydim, xdim, zdim = data_matrix_int.shape

    data_matrix = data_matrix_int.astype(np.uint8)
    print xdim, ydim, zdim

    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    midsection = (first_bs_sec + last_bs_sec)/2

    for l in labels[8:]:
#     for l in ['LRt']:

        print l

        vol_pair = (data_matrix == label_dict[l]).astype(np.float)
        
        vol_list = []
        name_list = []
        
        if l not in annotation_zlimits:
            continue
        
        if len(annotation_zlimits[l]) == 1:
            vol_list.append(vol_pair)
            name_list.append(l)
            
        elif len(annotation_zlimits[l]) == 2:
            vol = vol_pair.copy()

            lo, up = np.array(annotation_zlimits[l][0])*section_zdistance - volume_zmin
            print lo, up
            vol[..., :int(np.floor(lo))] = 0
            vol[..., int(np.ceil(up)):] = 0
            vol_list.append(vol)
            name_list.append(l+'_L')

            vol = vol_pair.copy()

            lo, up = np.array(annotation_zlimits[l][1])*section_zdistance - volume_zmin
            print lo, up            
            vol[..., :int(np.floor(lo))] = 0
            vol[..., int(np.ceil(up)):] = 0
            vol_list.append(vol)
            name_list.append(l+'_R')

        for vol, name in izip(vol_list, name_list):
                
            vs, faces = marching_cubes(vol, .5)
            fs = correct_mesh_orientation(vol, vs, faces)

            print vs.shape, fs.shape

            polydata = vtk.vtkPolyData()

            points = vtk.vtkPoints()
            for pt_ind, (x,y,z) in enumerate(vs):
                points.InsertPoint(pt_ind, x, y, z)

            cells = vtk.vtkCellArray()
            for ia, ib, ic in fs:
                cells.InsertNextCell(3)
                cells.InsertCellPoint(ia)
                cells.InsertCellPoint(ib)
                cells.InsertCellPoint(ic)

            polydata.SetPoints(points)
            polydata.SetPolys(cells)
            polydata.SetVerts(cells)

            for simplify_iter in range(3):

                deci = vtk.vtkQuadricDecimation()
                if simplify_iter == 0:
                    deci.SetInputData(polydata)
                else:
                    deci.SetInputData(smoother.GetOutput())
                deci.SetTargetReduction(0.8)
                deci.Update()

                smoother = vtk.vtkWindowedSincPolyDataFilter()
        #         smoother.NormalizeCoordinatesOn()
                smoother.SetPassBand(.1)
                smoother.SetNumberOfIterations(20)
                smoother.SetInputData(deci.GetOutput())
                smoother.Update()

                print deci.GetOutput().GetNumberOfPoints()

            stlWriter = vtk.vtkSTLWriter()

            stlWriter.SetFileName(mesh_dir + "/%(stack)s_%(name)s.stl" % {'stack': stack, 'name': name})

            stlWriter.SetInputData(smoother.GetOutput())
            stlWriter.Write()            

914 376 455
SuVe
VLL
6.26083 66.043432
394.847743 454.630345
(47580, 3) (95116, 3)
9533
1916
386
(53362, 3) (106486, 3)
10691
2158
437
6N
172.021681 199.195591
272.565148 299.739058
(5458, 3) (10906, 3)
1093
220
45
(6522, 3) (13036, 3)
1306
262
54
Amb
38.869522 52.456477
394.847743 405.717307
(5190, 3) (10372, 3)
1040
209
43
(4756, 3) (9508, 3)
952
192
40
R
136.695598 215.499937
275.282539 316.043404
(41920, 3) (83820, 3)
8388
1679
337
(27780, 3) (55554, 3)
5557
1112
224
Tz
125.826034 180.173854
272.565148 329.630359
(37314, 3) (74588, 3)
7478
1504
304
(30198, 3) (60364, 3)
6046
1209
243
Sol
(41592, 3) (83112, 3)
8337
1665
335
RtTg
LRt
19.847785 106.804297
335.065141 432.891217
(68692, 3) (137350, 3)
13747
2751
551
(71948, 3) (143868, 3)
14395
2879
577
LC
112.239079 136.695598
351.369487 378.543397
(19166, 3) (38314, 3)
3840
769
155
(15600, 3) (31176, 3)
3127
625
126
AP
(12636, 3) (25266, 3)
2528
507
103
sp5
6.26083 218.217328
242.673847 449.195563
(217896, 3) (435482, 3)
43664
8693
17

In [66]:
colors = np.random.randint(0, 255, (len(polydata_list), 3)) / 255.

In [70]:
np.savetxt('colors.txt', colors)

In [81]:
polydata_list = {}

for stack in ['MD593']:
    for l in labels[1:]:
        fn_L = '/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_L.stl' % {'stack': stack, 'label': l}
        
        if os.path.exists(fn_L):
            reader = vtk.vtkSTLReader()
            reader.SetFileName(fn_L)
            reader.Update()

            polydata_list[l] = []
            polydata_list[l].append(reader.GetOutput())

            fn_R = '/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s_R.stl' % {'stack': stack, 'label': l}
            reader = vtk.vtkSTLReader()
            reader.SetFileName(fn_R)
            reader.Update()

            polydata_list[l].append(reader.GetOutput())
            
        else:
            fn = '/home/yuncong/CSHL_meshes/%(stack)s/%(stack)s_%(label)s.stl' % {'stack': stack, 'label': l}
            
            reader = vtk.vtkSTLReader()
            reader.SetFileName(fn)
            reader.Update()
            
            polydata_list[l] = [reader.GetOutput()]

In [111]:
ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

camera = vtk.vtkCamera()

# camera.SetViewUp(-1, -1, 0)
# camera.SetPosition(-20, -30, 10)
# camera.SetFocalPoint(1, -1, -1)

# saggital
# camera.SetViewUp(-1, 0, 0)
# camera.SetPosition(0, 0, 2)
# camera.SetFocalPoint(0, 0, 1)

# horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(1, 0, 0)

# # coronal
# camera.SetViewUp(-1, 0, 0)
# camera.SetPosition(0, -3, 0)
# camera.SetFocalPoint(0, 1, 0)

for i, (l, polydatas) in enumerate(polydata_list.iteritems()):

    for polydata in polydatas:
    
        m = vtk.vtkPolyDataMapper()
        m.SetInputData(polydata)
    #     m.SetInputData(reader.GetOutput())
    #     m.SetInputConnection(reader.GetOutputPort())

        a = vtk.vtkActor()
        a.SetMapper(m)
#         a.GetProperty().SetRepresentationToWireframe()
        a.GetProperty().SetColor(colors[i])

        ren1.AddActor(a)
        ren1.SetActiveCamera(camera)
        ren1.ResetCamera()

renWin.Render()
iren.Start()

In [112]:
from collections import defaultdict

In [127]:
vertices_allLandmarks = defaultdict(list)
for i, (l, polydatas) in enumerate(polydata_list.iteritems()):
    for polydata in polydatas:
        try:
            vertices = numpy_support.vtk_to_numpy(polydata.GetPoints().GetData())
            vertices_allLandmarks[l].append(vertices)
        except Exception as e:
            print l
            print e.message

RtTg
'NoneType' object has no attribute 'GetData'
SuVe
'NoneType' object has no attribute 'GetData'


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [131]:
centroid_allLandmarks = dict([(l, [vs.mean(axis=0) for vs in vs_list]) 
                              for l, vs_list in vertices_allLandmarks.iteritems()])

In [132]:
centroid_allLandmarks

{'12N': [array([ 167.35136414,  544.12615967,  234.80662537], dtype=float32)],
 '5N': [array([ 129.01428223,  218.81552124,   50.86375046], dtype=float32),
  array([ 151.58226013,  230.10757446,  419.84210205], dtype=float32)],
 '6N': [array([ 153.94859314,  297.47265625,  185.3600769 ], dtype=float32),
  array([ 151.12915039,  299.28390503,  284.74005127], dtype=float32)],
 '7N': [array([ 256.95004272,  351.48596191,   55.20360565], dtype=float32),
  array([ 284.63525391,  351.98156738,  385.42202759], dtype=float32)],
 '7n': [array([ 171.3986969 ,  278.80847168,   93.85617065], dtype=float32),
  array([ 198.30714417,  279.4826355 ,  374.16506958], dtype=float32)],
 'AP': [array([ 110.82328796,  542.5501709 ,  239.81834412], dtype=float32)],
 'Amb': [array([ 211.80039978,  454.78070068,   44.19460297], dtype=float32),
  array([ 250.92416382,  454.76837158,  397.52661133], dtype=float32)],
 'Gr': [array([ 121.49416351,  604.61016846,  201.12780762], dtype=float32),
  array([ 135.192031